In [80]:
import pandas as pd

In [81]:
# no of questions marked,
# no of questions soft correct (0.5 for Y-S or N-S, 1 for Y-N) and hard correct, 

# no of easy questions marked,
# no of questions soft correct (0.5 for Y-S or N-S, 1 for Y-N) and hard correct,
# No of snippets for easy questions correctly marked

In [82]:
def preprocess(ans_df, res_df):
    num_ques = 5

    def add_id(df, key):
        df["uid"] = ""
        df["uid"] = df.uid.str.cat([df[key + str(i+1)] for i in range(num_ques)], '#')
        df.set_index("uid")
        return df

    ans_df = add_id(ans_df, "id")
    res_df = add_id(res_df, "Input.id")
    df = pd.merge(ans_df, res_df, left_index=True, right_index=True)
    # fill correct answer for answerable

    for i in range(num_ques):
        df["is_easy" + str(i+1)] = df["is_easy" + str(i + 1)].replace('Y', True)
        df["is_easy" + str(i+1)] = df["is_easy" + str(i + 1)].replace('N', False)

        df["is_answerable" + str(i + 1)] = df["is_answerable" + str(i + 1)].replace('Y', 1)
        df["is_answerable" + str(i + 1)] = df["is_answerable" + str(i + 1)].replace('S', 2)    
        df["is_answerable" + str(i + 1)] = df["is_answerable" + str(i + 1)].replace('N', 3)

        df["is_answerable_SY" + str(i + 1)] = df["is_answerable" + str(i + 1)].replace(2, 1)
        df["is_answerable_NY" + str(i + 1)] = df["is_answerable" + str(i + 1)].replace(3, 2)

        df["Answer.Q" + str(i + 1) + "ans_SY"] = df["Answer.Q" + str(i + 1) + "ans"].replace(2, 1)
        df["Answer.Q" + str(i + 1) + "ans_NY"] = df["Answer.Q" + str(i + 1) + "ans"].replace(3, 2)


    for i in range(num_ques):
        df["hard_check" + str(i+1)]    = df["is_answerable" + str(i + 1)] == df["Answer.Q" + str(i + 1) + "ans"]
        df["soft_check_SY" + str(i+1)] = df["is_answerable_SY" + str(i + 1)] == df["Answer.Q" + str(i + 1) + "ans_SY"]
        df["soft_check_NY" + str(i+1)] = df["is_answerable_NY" + str(i + 1)] == df["Answer.Q" + str(i + 1) + "ans_NY"]
    
    return df

In [193]:
def print_accuracy(df):
    
    # ALL ACCURACY
    
    L = len(df)
    TL = L * num_ques
    """
    print(sum([sum(df["hard_check"    + str(i + 1)]) for i in range(num_ques)]) / TL)
    print(sum([sum(df["soft_check_SY" + str(i + 1)]) for i in range(num_ques)]) / TL)
    print(sum([sum(df["soft_check_NY" + str(i + 1)]) for i in range(num_ques)]) / TL)

    print("-"*20)

    #EASY ACCURACY

    print(sum([sum(df["hard_check"    + str(i + 1)] & df["is_easy" + str(i + 1)]) for i in range(num_ques)]) / L)
    print(sum([sum(df["soft_check_SY" + str(i + 1)] & df["is_easy" + str(i + 1)]) for i in range(num_ques)]) / L)
    print(sum([sum(df["soft_check_NY" + str(i + 1)] & df["is_easy" + str(i + 1)]) for i in range(num_ques)]) / L)

    print("-"*20)
    
    #Accuracy after removing bad hits
    """
    def get_hits_to_keep(df, str1, str2):
        df[str1] = True
        for i in range(num_ques):
            df[str1] &= ~df["is_easy" + str(i + 1)] | df[str2 + str(i + 1)]
        return df

    df = get_hits_to_keep(df, "keep", "hard_check")
    df = get_hits_to_keep(df, "keep_SY", "soft_check_SY")            
    df = get_hits_to_keep(df, "keep_NY", "soft_check_NY")
    
    Lkeep = len(df[df["keep"] == True])
    LkeepSY = len(df[df["keep_SY"] == True])

    print(Lkeep/21., (sum([ sum(df["hard_check"    + str(i + 1)] & df["keep"])    for i in range(num_ques)]) - Lkeep)   / (4*Lkeep) )
    print(LkeepSY/21., (sum([ sum(df["soft_check_SY" + str(i + 1)] & df["keep_SY"]) for i in range(num_ques)]) - LkeepSY) / (4*LkeepSY) )
    #print(sum([ sum(df["soft_check_NY" + str(i + 1)] & df["keep_NY"]) for i in range(num_ques)]) - L / (4.*len(df[df["keep_NY"] == True])))

In [194]:
def prepreprocess(df):
    raw_df = pd.read_csv('gold_set_raw.csv')
    raw_df.set_index("question", inplace=True)

    for i in range(1, num_ques + 1):
        df["Input.id" + str(i)] = ""

    for i in range(1, num_ques + 1):
        for hit in range(len(df)):
            ques = df["Input.question" + str(i)].iloc[hit]
            df.at[hit, "Input.id" + str(i)] = raw_df.loc[ques]["id"]
    return df

In [195]:
print("15 cents")
res_df_15 = pd.read_csv('Batch_3431355_batch_results_15_cents.csv')
df_15 = preprocess(ans_df, res_df_15)
print_accuracy(df_15)

print("-"*40)

print("12 cents")
res_df_12 = pd.read_csv('Batch_3430899_batch_results_12_cents.csv')
res_df_12 = prepreprocess(res_df_12)
df_12 = preprocess(ans_df, res_df_12)
print_accuracy(df_12)

print("-"*40)

print("10 cents")
res_df_10 = pd.read_csv('Batch_3430691_batch_results_10_cents.csv')
res_df_10 = prepreprocess(res_df_10)
df_10 = preprocess(ans_df, res_df_10)
print_accuracy(df_10)



15 cents
0.8095238095238095 0.6470588235294118
0.8095238095238095 0.7352941176470589
----------------------------------------
12 cents
0.5714285714285714 0.4375
0.6666666666666666 0.6428571428571429
----------------------------------------
10 cents
0.6666666666666666 0.5535714285714286
0.7142857142857143 0.75


In [204]:
print("12 cents part 2")
res_df_12 = pd.read_csv('Batch_3432093_batch_results_12cents_part2.csv')
res_df_12 = prepreprocess(res_df_12)
df_12 = preprocess(ans_df, res_df_12)
print_accuracy(df_12)

print("-"*40)

print("10 cents part 2")
res_df_10 = pd.read_csv('Batch_3432101_batch_results_10cents_part2.csv')
res_df_10 = prepreprocess(res_df_10)
df_10 = preprocess(ans_df, res_df_10)
print_accuracy(df_10)

12 cents part 2
0.6666666666666666 0.625
0.7142857142857143 0.7666666666666667
----------------------------------------
10 cents part 2
0.7619047619047619 0.546875
0.8095238095238095 0.6911764705882353


In [197]:
df1 = pd.read_csv('data/electronics_10000.csv')
df2 = pd.read_csv('data/easy_data.csv')

In [198]:
df1.columns

Index(['Unnamed: 0', 'id', 'question', 'review0', 'review1', 'review2',
       'review3', 'review4'],
      dtype='object')

In [199]:
df2.columns

Index(['id', 'question', 'is_answerable', 'relevant_snippets', 'Snippet1',
       'Snippet2', 'Snippet3', 'Snippet4', 'Snippet5'],
      dtype='object')

In [202]:
df2.index

RangeIndex(start=0, stop=9, step=1)